In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data for Config 1
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Filter for Config 1 (10MW/10MWh) base case scenario
config_1_base = df[(df['config_id'] == 1) & (df['scenario'] == 'base_case')].iloc[0]

print("Using Config 1 (10MW/10MWh) base case data:")
print(f"DAM: mean=£{config_1_base['dam_mean']:.2f}, stdev=£{config_1_base['dam_std']:.2f}")
print(f"IDM: mean=£{config_1_base['idm_mean']:.2f}, stdev=£{config_1_base['idm_std']:.2f}")
print(f"DC: mean=£{config_1_base['dc_mean']:.2f}, stdev=£{config_1_base['dc_std']:.2f}")
print(f"Correlations: DAM-IDM={config_1_base['corr_dam_idm']:.3f}, DAM-DC={config_1_base['corr_dam_dc']:.3f}, IDM-DC={config_1_base['corr_idm_dc']:.3f}")

# Extract market data from CSV
returns_original = np.array([config_1_base['dam_mean'], config_1_base['idm_mean'], config_1_base['dc_mean']])

risks_original = np.array([config_1_base['dam_std'], config_1_base['idm_std'], config_1_base['dc_std']])

# Normalize returns and risks (mean=0, std=1)
returns = (returns_original - returns_original.mean()) / returns_original.std()
risks = (risks_original - risks_original.mean()) / risks_original.std()

# Build correlation matrix from CSV data
correlations = np.array([
    [1.000, config_1_base['corr_dam_idm'], config_1_base['corr_dam_dc']],
    [config_1_base['corr_dam_idm'], 1.000, config_1_base['corr_idm_dc']],
    [config_1_base['corr_dam_dc'], config_1_base['corr_idm_dc'], 1.000]
])

# Calculate covariance matrix with normalized data
cov_matrix = np.outer(risks, risks) * correlations

def calculate_portfolio_stats(weights):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio():
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(n_portfolios=100):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio()
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights)
    
    return best_weights, best_return, best_risk

# Generate efficient frontier
efficient_frontier = generate_efficient_frontier(100)

# Create interactive plot
fig = go.Figure()

# Plot efficient frontier
if len(efficient_frontier) > 0:
    fig.add_trace(go.Scatter(
        x=efficient_frontier[:, 1],  # Risk (normalized)
        y=efficient_frontier[:, 0],  # Return (normalized)
        mode='lines+markers',
        marker=dict(size=4, color='blue', opacity=0.7),
        line=dict(color='blue', width=2),
        name='Efficient Frontier',
        hovertemplate='<b>Efficient Portfolio:</b><br>' +
                      'DAM: %{customdata[0]:.1%}<br>' +
                      'IDM: %{customdata[1]:.1%}<br>' +
                      'DC: %{customdata[2]:.1%}<br>' +
                      '<b>Original Scale:</b><br>' +
                      'Return: £%{customdata[3]:.2f}<br>' +
                      'Risk: £%{customdata[4]:.2f}<br>' +
                      '<extra></extra>',
        customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                   efficient_frontier[:, 3],  # IDM weight
                                   efficient_frontier[:, 4],  # DC weight
                                   efficient_frontier[:, 5],  # Original return
                                   efficient_frontier[:, 6])) # Original risk
    ))

# Find and plot optimal portfolio for A=3
A = 3
weights, expected_return, risk = find_best_portfolio(A)
_, _, variance = calculate_portfolio_stats(weights)
opt_utility = expected_return - 0.5 * A * variance

# Convert to original scale
original_return = expected_return * returns_original.std() + returns_original.mean()
original_risk = risk * risks_original.std() + risks_original.mean()

# Create indifference curve: E[R] = U + 0.5*A*σ²
risk_range = np.linspace(0, max(efficient_frontier[:, 1]) if len(efficient_frontier) > 0 else 2, 200)
variance_range = risk_range ** 2
indifference_curve = opt_utility + 0.5 * A * variance_range

# Plot indifference curve
fig.add_trace(go.Scatter(
    x=risk_range,
    y=indifference_curve,
    mode='lines',
    line=dict(color='red', dash='dash', width=3),
    name=f'Indifference Curve (A={A})',
    hovertemplate='<b>Utility Function:</b><br>' +
                  'U = E(r) - 0.5 × ' + str(A) + ' × σ²<br>' +
                  '<extra></extra>'
))

# Plot optimal portfolio
fig.add_trace(go.Scatter(
    x=[risk],
    y=[expected_return],
    mode='markers',
    marker=dict(symbol='x', size=15, color='red', line=dict(width=2)),
    name=f'Optimal Portfolio (A={A})',
    hovertemplate='<b>Optimal Portfolio (A=' + str(A) + '):</b><br>' +
                  'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                  'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                  'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                  '<b>Original Scale:</b><br>' +
                  'Return: £' + f'{original_return:.2f}' + '<br>' +
                  'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                  '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                  '<extra></extra>'
))

# Plot individual assets for reference
asset_names = ['DAM', 'IDM', 'DC']
asset_colors = ['green', 'orange', 'purple']
asset_symbols = ['circle', 'square', 'triangle-up']

for i, (name, color, symbol) in enumerate(zip(asset_names, asset_colors, asset_symbols)):
    # Individual asset has 100% weight in that asset, 0% in others
    asset_weights = np.zeros(3)
    asset_weights[i] = 1.0
    asset_return, asset_risk, _ = calculate_portfolio_stats(asset_weights)
    
    # Convert to original scale
    asset_orig_return = asset_return * returns_original.std() + returns_original.mean()
    asset_orig_risk = asset_risk * risks_original.std() + risks_original.mean()
    
    fig.add_trace(go.Scatter(
        x=[asset_risk],
        y=[asset_return],
        mode='markers',
        marker=dict(symbol=symbol, size=12, color=color, line=dict(width=2, color='black')),
        name=f'{name} (100%)',
        hovertemplate='<b>' + name + ' (100%):</b><br>' +
                      'Return: £' + f'{asset_orig_return:.2f}' + '<br>' +
                      'Risk: £' + f'{asset_orig_risk:.2f}' + '<br>' +
                      '<extra></extra>'
    ))

# Update layout
fig.update_layout(
    title='Config 1 (10MW/10MWh): Efficient Frontier - DAM, IDM, DC Markets<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1000,
    height=700,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98)
)

fig.show(renderer="browser")

# Print results
print("\n" + "="*60)
print("EFFICIENT FRONTIER ANALYSIS")
print("="*60)

print(f"\nOptimal Portfolio (A={A}):")
print(f"  DAM:     {weights[0]:6.1%}")
print(f"  IDM:     {weights[1]:6.1%}")
print(f"  DC:      {weights[2]:6.1%}")
print(f"  Expected Return: £{original_return:7.2f}")
print(f"  Portfolio Risk:  £{original_risk:7.2f}")
print(f"  Utility (A=3):   {opt_utility:7.3f}")

if len(efficient_frontier) > 0:
    # Find minimum risk portfolio from efficient frontier
    min_risk_idx = np.argmin(efficient_frontier[:, 1])
    min_risk_ef = efficient_frontier[min_risk_idx]
    
    print(f"\nMinimum Risk Portfolio:")
    print(f"  DAM:     {min_risk_ef[2]:6.1%}")
    print(f"  IDM:     {min_risk_ef[3]:6.1%}")
    print(f"  DC:      {min_risk_ef[4]:6.1%}")
    print(f"  Expected Return: £{min_risk_ef[5]:7.2f}")
    print(f"  Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
    
    print(f"\nEfficient Frontier contains {len(efficient_frontier)} portfolios")
    print(f"Risk range: £{min(efficient_frontier[:, 6]):.2f} - £{max(efficient_frontier[:, 6]):.2f}")
    print(f"Return range: £{min(efficient_frontier[:, 5]):.2f} - £{max(efficient_frontier[:, 5]):.2f}")

Using Config 1 (10MW/10MWh) base case data:
DAM: mean=£446.89, stdev=£312.07
IDM: mean=£472.66, stdev=£389.16
DC: mean=£797.87, stdev=£403.13
Correlations: DAM-IDM=0.665, DAM-DC=0.723, IDM-DC=0.476

EFFICIENT FRONTIER ANALYSIS

Optimal Portfolio (A=3):
  DAM:      20.4%
  IDM:       0.0%
  DC:       79.6%
  Expected Return: £ 726.27
  Portfolio Risk:  £ 389.26
  Utility (A=3):     0.545

Minimum Risk Portfolio:
  DAM:      26.4%
  IDM:      49.4%
  DC:       24.3%
  Expected Return: £ 544.84
  Portfolio Risk:  £ 377.97

Efficient Frontier contains 100 portfolios
Risk range: £377.97 - £403.13
Return range: £544.84 - £797.87


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 1 and current scenario
    config_data = df[(df['config_id'] == 1) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_1_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_1_scenario['dam_mean'], config_1_scenario['idm_mean'], config_1_scenario['dc_mean']])
    risks_original = np.array([config_1_scenario['dam_std'], config_1_scenario['idm_std'], config_1_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_1_scenario['corr_dam_idm'], config_1_scenario['corr_dam_dc']],
        [config_1_scenario['corr_dam_idm'], 1.000, config_1_scenario['corr_idm_dc']],
        [config_1_scenario['corr_dam_dc'], config_1_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Plot efficient frontier
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 1 (10MW/10MWh): Efficient Frontiers for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      20.6%
    IDM:       0.0%
    DC:       79.4%
    Expected Return: £ 915.56
    Portfolio Risk:  £ 389.12
  Minimum Risk Portfolio:
    DAM:      26.4%
    IDM:      49.3%
    DC:       24.3%
    Expected Return: £ 603.14
    Portfolio Risk:  £ 377.96
  Efficient Frontier:
    Portfolios: 50
    Risk range: £377.96 - £403.10
    Return range: £603.14 - £1037.24

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      20.6%
    IDM:       0.0%
    DC:       79.4%
    Expected Return: £ 852.45
    Portfolio Risk:  £ 389.16
  Minimum Risk Portfolio:
    DAM:      26.4%
    IDM:      49.3%
    DC:       24.3%
    Expected Return: £ 583.68
    Portfolio Risk:  £ 377.96
  Efficient Frontier:
    Portfolios: 50
    Risk range: £377.96 - £403.11
    Return range: £583.68 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      20.5%
    IDM:       0.0%
    DC:       79.5%
    

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 1 and current scenario
    config_data = df[(df['config_id'] == 1) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_1_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_1_scenario['dam_mean'], config_1_scenario['idm_mean'], config_1_scenario['dc_mean']])
    risks_original = np.array([config_1_scenario['dam_std'], config_1_scenario['idm_std'], config_1_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_1_scenario['corr_dam_idm'], config_1_scenario['corr_dam_dc']],
        [config_1_scenario['corr_dam_idm'], 1.000, config_1_scenario['corr_idm_dc']],
        [config_1_scenario['corr_dam_dc'], config_1_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 1 (10MW/10MWh): Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      20.6%
    IDM:       0.0%
    DC:       79.4%
    Expected Return: £ 915.56
    Portfolio Risk:  £ 389.12
    Utility (A=3):     0.553
  Minimum Risk Portfolio:
    DAM:      26.4%
    IDM:      49.3%
    DC:       24.3%
    Expected Return: £ 603.14
    Portfolio Risk:  £ 377.96
  Efficient Frontier:
    Portfolios: 50
    Risk range: £377.96 - £403.10
    Return range: £603.14 - £1037.24

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      20.6%
    IDM:       0.0%
    DC:       79.4%
    Expected Return: £ 852.45
    Portfolio Risk:  £ 389.16
    Utility (A=3):     0.551
  Minimum Risk Portfolio:
    DAM:      26.4%
    IDM:      49.3%
    DC:       24.3%
    Expected Return: £ 583.68
    Portfolio Risk:  £ 377.96
  Efficient Frontier:
    Portfolios: 50
    Risk range: £377.96 - £403.11
    Return range: £583.68 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A=3):
    DA

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 8 and current scenario
    config_data = df[(df['config_id'] == 8) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_8_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_8_scenario['dam_mean'], config_8_scenario['idm_mean'], config_8_scenario['dc_mean']])
    risks_original = np.array([config_8_scenario['dam_std'], config_8_scenario['idm_std'], config_8_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_8_scenario['corr_dam_idm'], config_8_scenario['corr_dam_dc']],
        [config_8_scenario['corr_dam_idm'], 1.000, config_8_scenario['corr_idm_dc']],
        [config_8_scenario['corr_dam_dc'], config_8_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 8: Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 8")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 8

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      39.5%
    DC:       60.5%
    Expected Return: £ 976.89
    Portfolio Risk:  £ 691.85
    Utility (A=3):    -0.064
  Minimum Risk Portfolio:
    DAM:      92.9%
    IDM:       0.0%
    DC:        7.1%
    Expected Return: £ 859.27
    Portfolio Risk:  £ 602.58
  Efficient Frontier:
    Portfolios: 50
    Risk range: £602.58 - £777.91
    Return range: £859.27 - £1037.24

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      41.5%
    DC:       58.5%
    Expected Return: £ 927.07
    Portfolio Risk:  £ 689.34
    Utility (A=3):    -0.013
  Minimum Risk Portfolio:
    DAM:      92.9%
    IDM:       0.0%
    DC:        7.1%
    Expected Return: £ 853.60
    Portfolio Risk:  £ 602.58
  Efficient Frontier:
    Portfolios: 50
    Risk range: £602.58 - £777.91
    Return range: £853.60 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 5 and current scenario
    config_data = df[(df['config_id'] == 5) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_5_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_5_scenario['dam_mean'], config_5_scenario['idm_mean'], config_5_scenario['dc_mean']])
    risks_original = np.array([config_5_scenario['dam_std'], config_5_scenario['idm_std'], config_5_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_5_scenario['corr_dam_idm'], config_5_scenario['corr_dam_dc']],
        [config_5_scenario['corr_dam_idm'], 1.000, config_5_scenario['corr_idm_dc']],
        [config_5_scenario['corr_dam_dc'], config_5_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 5: Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 5")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 5

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      26.9%
    DC:       73.1%
    Expected Return: £ 941.60
    Portfolio Risk:  £ 529.23
    Utility (A=3):     0.146
  Minimum Risk Portfolio:
    DAM:      87.0%
    IDM:      13.0%
    DC:        0.0%
    Expected Return: £ 653.35
    Portfolio Risk:  £ 491.90
  Efficient Frontier:
    Portfolios: 50
    Risk range: £491.90 - £556.42
    Return range: £653.35 - £1037.24

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      27.1%
    DC:       72.9%
    Expected Return: £ 882.64
    Portfolio Risk:  £ 529.08
    Utility (A=3):     0.152
  Minimum Risk Portfolio:
    DAM:      87.0%
    IDM:      13.0%
    DC:        0.0%
    Expected Return: £ 653.35
    Portfolio Risk:  £ 491.91
  Efficient Frontier:
    Portfolios: 50
    Risk range: £491.91 - £556.41
    Return range: £653.35 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 15 and current scenario
    config_data = df[(df['config_id'] == 15) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_15_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_15_scenario['dam_mean'], config_15_scenario['idm_mean'], config_15_scenario['dc_mean']])
    risks_original = np.array([config_15_scenario['dam_std'], config_15_scenario['idm_std'], config_15_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_15_scenario['corr_dam_idm'], config_15_scenario['corr_dam_dc']],
        [config_15_scenario['corr_dam_idm'], 1.000, config_15_scenario['corr_idm_dc']],
        [config_15_scenario['corr_dam_dc'], config_15_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 15: Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 15")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 15

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      90.4%
    IDM:       9.6%
    DC:        0.0%
    Expected Return: £ 596.24
    Portfolio Risk:  £ 445.47
    Utility (A=3):     0.214
  Minimum Risk Portfolio:
    DAM:      81.0%
    IDM:       0.0%
    DC:       19.0%
    Expected Return: £ 579.81
    Portfolio Risk:  £ 418.30
  Efficient Frontier:
    Portfolios: 50
    Risk range: £418.30 - £521.89
    Return range: £579.81 - £615.85

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      99.5%
    IDM:       0.5%
    DC:        0.0%
    Expected Return: £ 594.25
    Portfolio Risk:  £ 440.95
    Utility (A=3):     0.294
  Minimum Risk Portfolio:
    DAM:      81.0%
    IDM:       0.0%
    DC:       19.0%
    Expected Return: £ 572.24
    Portfolio Risk:  £ 418.30
  Efficient Frontier:
    Portfolios: 50
    Risk range: £418.30 - £521.89
    Return range: £572.24 - £615.85

UP_10% SCENARIO:
  Optimal Portfolio (A

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 17 and current scenario
    config_data = df[(df['config_id'] == 17) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_17_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_17_scenario['dam_mean'], config_17_scenario['idm_mean'], config_17_scenario['dc_mean']])
    risks_original = np.array([config_17_scenario['dam_std'], config_17_scenario['idm_std'], config_17_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_17_scenario['corr_dam_idm'], config_17_scenario['corr_dam_dc']],
        [config_17_scenario['corr_dam_idm'], 1.000, config_17_scenario['corr_idm_dc']],
        [config_17_scenario['corr_dam_dc'], config_17_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 17: Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 17")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 17

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      98.5%
    IDM:       0.0%
    DC:        1.5%
    Expected Return: £2181.12
    Portfolio Risk:  £1673.10
    Utility (A=3):     0.230
  Minimum Risk Portfolio:
    DAM:      77.3%
    IDM:       0.0%
    DC:       22.7%
    Expected Return: £2046.80
    Portfolio Risk:  £1562.46
  Efficient Frontier:
    Portfolios: 50
    Risk range: £1562.46 - £1932.76
    Return range: £2046.80 - £2271.53

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      98.7%
    IDM:       0.0%
    DC:        1.3%
    Expected Return: £2181.22
    Portfolio Risk:  £1675.19
    Utility (A=3):     0.249
  Minimum Risk Portfolio:
    DAM:      77.3%
    IDM:       0.0%
    DC:       22.7%
    Expected Return: £2019.67
    Portfolio Risk:  £1562.47
  Efficient Frontier:
    Portfolios: 50
    Risk range: £1562.47 - £1932.76
    Return range: £2019.67 - £2271.53

UP_10% SCENARIO:
  Optimal Port

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D*_price_addition_profit_stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 17 and current scenario
    config_data = df[(df['config_id'] == 17) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_17_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_17_scenario['dam_mean'], config_17_scenario['idm_mean'], config_17_scenario['dc_mean']])
    risks_original = np.array([config_17_scenario['dam_std'], config_17_scenario['idm_std'], config_17_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_17_scenario['corr_dam_idm'], config_17_scenario['corr_dam_dc']],
        [config_17_scenario['corr_dam_idm'], 1.000, config_17_scenario['corr_idm_dc']],
        [config_17_scenario['corr_dam_dc'], config_17_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier as LINE
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            showlegend=True,
            hoverinfo='skip'  # Skip hover for the line
        ))
        
        # Plot efficient frontier as DOTS with hover information
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='markers',
            marker=dict(size=4, color=scenario_colors[i], opacity=0.8),
            name=f'Efficient Portfolios ({scenario})',
            showlegend=False,  # Don't show in legend to avoid clutter
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 17: Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 17")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 17

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      98.5%
    IDM:       0.0%
    DC:        1.5%
    Expected Return: £2181.12
    Portfolio Risk:  £1673.10
    Utility (A=3):     0.230
  Minimum Risk Portfolio:
    DAM:      77.3%
    IDM:       0.0%
    DC:       22.7%
    Expected Return: £2046.80
    Portfolio Risk:  £1562.46
  Efficient Frontier:
    Portfolios: 50
    Risk range: £1562.46 - £1932.76
    Return range: £2046.80 - £2271.53

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      98.7%
    IDM:       0.0%
    DC:        1.3%
    Expected Return: £2181.22
    Portfolio Risk:  £1675.19
    Utility (A=3):     0.249
  Minimum Risk Portfolio:
    DAM:      77.3%
    IDM:       0.0%
    DC:       22.7%
    Expected Return: £2019.67
    Portfolio Risk:  £1562.47
  Efficient Frontier:
    Portfolios: 50
    Risk range: £1562.47 - £1932.76
    Return range: £2019.67 - £2271.53

UP_10% SCENARIO:
  Optimal Port

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D* test profits stats.csv')

# Define scenarios to analyze (using the actual scenario names from the CSV)
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 1 and current scenario
    config_data = df[(df['config_id'] == 1) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_1_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_1_scenario['dam_mean'], config_1_scenario['idm_mean'], config_1_scenario['dc_mean']])
    risks_original = np.array([config_1_scenario['dam_std'], config_1_scenario['idm_std'], config_1_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_1_scenario['corr_dam_idm'], config_1_scenario['corr_dam_dc']],
        [config_1_scenario['corr_dam_idm'], 1.000, config_1_scenario['corr_idm_dc']],
        [config_1_scenario['corr_dam_dc'], config_1_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier as LINE
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            showlegend=True,
            hoverinfo='skip'  # Skip hover for the line
        ))
        
        # Plot efficient frontier as DOTS with hover information
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='markers',
            marker=dict(size=4, color=scenario_colors[i], opacity=0.8),
            name=f'Efficient Portfolios ({scenario})',
            showlegend=False,  # Don't show in legend to avoid clutter
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 1 (10MW/10MWh): Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 1")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 1

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      41.0%
    IDM:       0.0%
    DC:       59.0%
    Expected Return: £ 795.40
    Portfolio Risk:  £ 456.65
    Utility (A=3):     0.071
  Minimum Risk Portfolio:
    DAM:       0.0%
    IDM:      91.0%
    DC:        9.0%
    Expected Return: £ 523.63
    Portfolio Risk:  £ 424.00
  Efficient Frontier:
    Portfolios: 50
    Risk range: £424.00 - £524.07
    Return range: £523.63 - £1037.23

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      37.9%
    IDM:       0.0%
    DC:       62.1%
    Expected Return: £ 763.95
    Portfolio Risk:  £ 434.02
    Utility (A=3):     0.125
  Minimum Risk Portfolio:
    DAM:       0.0%
    IDM:      96.7%
    DC:        3.3%
    Expected Return: £ 488.85
    Portfolio Risk:  £ 399.97
  Efficient Frontier:
    Portfolios: 50
    Risk range: £399.97 - £483.76
    Return range: £488.85 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D* test profits stats.csv')

# Define scenarios to analyze (using the actual scenario names from the CSV)
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 5 and current scenario
    config_data = df[(df['config_id'] == 5) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_5_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_5_scenario['dam_mean'], config_5_scenario['idm_mean'], config_5_scenario['dc_mean']])
    risks_original = np.array([config_5_scenario['dam_std'], config_5_scenario['idm_std'], config_5_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_5_scenario['corr_dam_idm'], config_5_scenario['corr_dam_dc']],
        [config_5_scenario['corr_dam_idm'], 1.000, config_5_scenario['corr_idm_dc']],
        [config_5_scenario['corr_dam_dc'], config_5_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier as LINE
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            showlegend=True,
            hoverinfo='skip'  # Skip hover for the line
        ))
        
        # Plot efficient frontier as DOTS with hover information
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='markers',
            marker=dict(size=4, color=scenario_colors[i], opacity=0.8),
            name=f'Efficient Portfolios ({scenario})',
            showlegend=False,  # Don't show in legend to avoid clutter
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 5: Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 5")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 5

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:       0.0%
    DC:      100.0%
    Expected Return: £1037.23
    Portfolio Risk:  £ 524.07
    Utility (A=3):     1.399
  Minimum Risk Portfolio:
    DAM:       5.3%
    IDM:       0.8%
    DC:       93.9%
    Expected Return: £1013.71
    Portfolio Risk:  £ 522.69
  Efficient Frontier:
    Portfolios: 50
    Risk range: £522.69 - £524.07
    Return range: £1013.71 - £1037.23

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:       0.0%
    DC:      100.0%
    Expected Return: £ 957.44
    Portfolio Risk:  £ 529.53
    Utility (A=3):     1.084
  Minimum Risk Portfolio:
    DAM:       9.6%
    IDM:      21.1%
    DC:       69.3%
    Expected Return: £ 869.64
    Portfolio Risk:  £ 523.37
  Efficient Frontier:
    Portfolios: 50
    Risk range: £523.37 - £529.53
    Return range: £869.64 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D* test profits stats.csv')

# Define scenarios to analyze (using the actual scenario names from the CSV)
scenarios = ['up_30%', 'up_20%', 'up_10%', 'down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=50):
    """Generate efficient frontier analytically"""
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    
    # Create range of target returns from minimum variance to maximum return
    target_returns = np.linspace(min_return, max_ret, n_portfolios)
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 8 and current scenario
    config_data = df[(df['config_id'] == 8) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_8_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_8_scenario['dam_mean'], config_8_scenario['idm_mean'], config_8_scenario['dc_mean']])
    risks_original = np.array([config_8_scenario['dam_std'], config_8_scenario['idm_std'], config_8_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_8_scenario['corr_dam_idm'], config_8_scenario['corr_dam_dc']],
        [config_8_scenario['corr_dam_idm'], 1.000, config_8_scenario['corr_idm_dc']],
        [config_8_scenario['corr_dam_dc'], config_8_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate efficient frontier for this scenario
    efficient_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(efficient_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(efficient_frontier[:, 1])
        
        # Plot efficient frontier as LINE
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='lines',
            line=dict(color=scenario_colors[i], width=2),
            name=f'Efficient Frontier ({scenario})',
            showlegend=True,
            hoverinfo='skip'  # Skip hover for the line
        ))
        
        # Plot efficient frontier as DOTS with hover information
        fig.add_trace(go.Scatter(
            x=efficient_frontier[:, 1],  # Risk (normalized)
            y=efficient_frontier[:, 0],  # Return (normalized)
            mode='markers',
            marker=dict(size=4, color=scenario_colors[i], opacity=0.8),
            name=f'Efficient Portfolios ({scenario})',
            showlegend=False,  # Don't show in legend to avoid clutter
            hovertemplate='<b>Efficient Portfolio (' + scenario + '):</b><br>' +
                          'DAM: %{customdata[0]:.1%}<br>' +
                          'IDM: %{customdata[1]:.1%}<br>' +
                          'DC: %{customdata[2]:.1%}<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £%{customdata[3]:.2f}<br>' +
                          'Risk: £%{customdata[4]:.2f}<br>' +
                          '<extra></extra>',
            customdata=np.column_stack((efficient_frontier[:, 2],  # DAM weight
                                       efficient_frontier[:, 3],  # IDM weight
                                       efficient_frontier[:, 4],  # DC weight
                                       efficient_frontier[:, 5],  # Original return
                                       efficient_frontier[:, 6])) # Original risk
        ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        # Use broader risk range to show utility curve properly
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)  # Extend range for better visualization
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,  # Don't clutter legend
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='x', size=12, color=scenario_colors[i], line=dict(width=2)),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': efficient_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 8: Efficient Frontiers & Utility Curves for Different DC Scenarios<br><sub>Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 8")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Find minimum risk portfolio from efficient frontier
        ef = data['efficient_frontier']
        min_risk_idx = np.argmin(ef[:, 1])
        min_risk_ef = ef[min_risk_idx]
        
        print(f"  Minimum Risk Portfolio:")
        print(f"    DAM:     {min_risk_ef[2]:6.1%}")
        print(f"    IDM:     {min_risk_ef[3]:6.1%}")
        print(f"    DC:      {min_risk_ef[4]:6.1%}")
        print(f"    Expected Return: £{min_risk_ef[5]:7.2f}")
        print(f"    Portfolio Risk:  £{min_risk_ef[6]:7.2f}")
        
        print(f"  Efficient Frontier:")
        print(f"    Portfolios: {len(ef)}")
        print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
        print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 8

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      29.4%
    DC:       70.6%
    Expected Return: £ 992.25
    Portfolio Risk:  £ 697.58
    Utility (A=3):     0.137
  Minimum Risk Portfolio:
    DAM:      88.7%
    IDM:      11.3%
    DC:        0.0%
    Expected Return: £ 850.00
    Portfolio Risk:  £ 645.15
  Efficient Frontier:
    Portfolios: 50
    Risk range: £645.15 - £737.59
    Return range: £850.00 - £1037.23

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      36.0%
    DC:       64.0%
    Expected Return: £ 931.13
    Portfolio Risk:  £ 693.93
    Utility (A=3):     0.069
  Minimum Risk Portfolio:
    DAM:      96.1%
    IDM:       3.9%
    DC:        0.0%
    Expected Return: £ 847.15
    Portfolio Risk:  £ 623.22
  Efficient Frontier:
    Portfolios: 50
    Risk range: £623.22 - £751.02
    Return range: £847.15 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A


COMPLETE EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 1

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      41.0%
    IDM:       0.0%
    DC:       59.0%
    Expected Return: £ 795.40
    Portfolio Risk:  £ 456.65
    Utility (A=3):     0.071
  Minimum Variance Portfolio:
    DAM:       0.0%
    IDM:      91.0%
    DC:        9.0%
    Expected Return: £ 523.63
    Portfolio Risk:  £ 424.00
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolios: 49
    Inefficient Portfolios: 50
    Risk range: £424.00 - £524.07
    Return range: £446.89 - £1037.23

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      37.9%
    IDM:       0.0%
    DC:       62.1%
    Expected Return: £ 763.95
    Portfolio Risk:  £ 434.02
    Utility (A=3):     0.125
  Minimum Variance Portfolio:
    DAM:       0.0%
    IDM:      96.7%
    DC:        3.3%
    Expected Return: £ 488.85
    Portfolio Risk:  £ 399.97
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolio

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D* test profits stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'base_case','down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'green','lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=100):
    """Generate complete efficient frontier including both efficient and inefficient portions"""
    
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    min_ret = min(returns)
    
    # Create extended range that goes beyond minimum variance portfolio
    # This creates the full hyperbola, not just the efficient portion
    return_range_low = np.linspace(min_ret, min_return, n_portfolios//2)  # Inefficient portion
    return_range_high = np.linspace(min_return, max_ret, n_portfolios//2)  # Efficient portion
    
    # Combine both ranges, removing duplicate minimum variance point
    target_returns = np.concatenate([return_range_low[:-1], return_range_high])
    
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            # Mark whether this is efficient (above min var) or inefficient (below min var)
            is_efficient = portfolio_return >= min_return
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk, is_efficient])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 1 and current scenario
    config_data = df[(df['config_id'] == 1) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_1_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_1_scenario['dam_mean'], config_1_scenario['idm_mean'], config_1_scenario['dc_mean']])
    risks_original = np.array([config_1_scenario['dam_std'], config_1_scenario['idm_std'], config_1_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_1_scenario['corr_dam_idm'], config_1_scenario['corr_dam_dc']],
        [config_1_scenario['corr_dam_idm'], 1.000, config_1_scenario['corr_idm_dc']],
        [config_1_scenario['corr_dam_dc'], config_1_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate complete efficient frontier for this scenario
    complete_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(complete_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(complete_frontier[:, 1])
        
        # Separate efficient and inefficient portions
        efficient_mask = complete_frontier[:, 7].astype(bool)  # is_efficient flag
        efficient_portion = complete_frontier[efficient_mask]
        inefficient_portion = complete_frontier[~efficient_mask]
        
        # Plot EFFICIENT portion as solid line
        if len(efficient_portion) > 0:
            fig.add_trace(go.Scatter(
                x=efficient_portion[:, 1],  # Risk (normalized)
                y=efficient_portion[:, 0],  # Return (normalized)
                mode='lines',
                line=dict(color=scenario_colors[i], width=3),
                name=f'Efficient Frontier ({scenario})',
                showlegend=True,
                hoverinfo='skip'
            ))
            
            # Add dots for efficient portion
            fig.add_trace(go.Scatter(
                x=efficient_portion[:, 1],
                y=efficient_portion[:, 0],
                mode='markers',
                marker=dict(size=4, color=scenario_colors[i], opacity=0.8),
                name=f'Efficient Portfolios ({scenario})',
                showlegend=False,
                hovertemplate='<b>EFFICIENT Portfolio (' + scenario + '):</b><br>' +
                              'DAM: %{customdata[0]:.1%}<br>' +
                              'IDM: %{customdata[1]:.1%}<br>' +
                              'DC: %{customdata[2]:.1%}<br>' +
                              '<b>Original Scale:</b><br>' +
                              'Return: £%{customdata[3]:.2f}<br>' +
                              'Risk: £%{customdata[4]:.2f}<br>' +
                              '<extra></extra>',
                customdata=np.column_stack((efficient_portion[:, 2],  # DAM weight
                                           efficient_portion[:, 3],  # IDM weight
                                           efficient_portion[:, 4],  # DC weight
                                           efficient_portion[:, 5],  # Original return
                                           efficient_portion[:, 6])) # Original risk
            ))
        
        # Plot INEFFICIENT portion as dashed line
        if len(inefficient_portion) > 0:
            fig.add_trace(go.Scatter(
                x=inefficient_portion[:, 1],  # Risk (normalized)
                y=inefficient_portion[:, 0],  # Return (normalized)
                mode='lines',
                line=dict(color=scenario_colors[i], width=2, dash='dash'),
                name=f'Inefficient Frontier ({scenario})',
                showlegend=False,  # Don't clutter legend
                hoverinfo='skip'
            ))
            
            # Add dots for inefficient portion
            fig.add_trace(go.Scatter(
                x=inefficient_portion[:, 1],
                y=inefficient_portion[:, 0],
                mode='markers',
                marker=dict(size=3, color=scenario_colors[i], opacity=0.5, symbol='circle-open'),
                name=f'Inefficient Portfolios ({scenario})',
                showlegend=False,
                hovertemplate='<b>INEFFICIENT Portfolio (' + scenario + '):</b><br>' +
                              'DAM: %{customdata[0]:.1%}<br>' +
                              'IDM: %{customdata[1]:.1%}<br>' +
                              'DC: %{customdata[2]:.1%}<br>' +
                              '<b>Original Scale:</b><br>' +
                              'Return: £%{customdata[3]:.2f}<br>' +
                              'Risk: £%{customdata[4]:.2f}<br>' +
                              '<i>Note: Higher risk for same return</i><br>' +
                              '<extra></extra>',
                customdata=np.column_stack((inefficient_portion[:, 2],  # DAM weight
                                           inefficient_portion[:, 3],  # IDM weight
                                           inefficient_portion[:, 4],  # DC weight
                                           inefficient_portion[:, 5],  # Original return
                                           inefficient_portion[:, 6])) # Original risk
            ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio using circle marker (dot)
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='circle', size=10, color=scenario_colors[i], line=dict(width=2, color='white')),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data (without min variance portfolio info since we're not displaying it)
        scenario_data[scenario] = {
            'efficient_frontier': complete_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 1: COMPLETE Efficient Frontiers for Different DC Scenarios<br><sub>Showing Full Hyperbola: Efficient (solid) & Inefficient (dashed) Portions | Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results (without minimum variance portfolio details)
print("\n" + "="*80)
print("COMPLETE EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 1")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Count efficient vs inefficient portfolios
        ef = data['efficient_frontier']
        if len(ef) > 0 and ef.shape[1] > 7:
            efficient_count = np.sum(ef[:, 7].astype(bool))
            inefficient_count = len(ef) - efficient_count
            
            print(f"  Complete Frontier:")
            print(f"    Total Portfolios: {len(ef)}")
            print(f"    Efficient Portfolios: {efficient_count}")
            print(f"    Inefficient Portfolios: {inefficient_count}")
            print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
            print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


COMPLETE EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 1

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      41.0%
    IDM:       0.0%
    DC:       59.0%
    Expected Return: £ 795.40
    Portfolio Risk:  £ 456.65
    Utility (A=3):     0.071
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolios: 49
    Inefficient Portfolios: 50
    Risk range: £424.00 - £524.07
    Return range: £446.89 - £1037.23

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      37.9%
    IDM:       0.0%
    DC:       62.1%
    Expected Return: £ 763.95
    Portfolio Risk:  £ 434.02
    Utility (A=3):     0.125
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolios: 49
    Inefficient Portfolios: 50
    Risk range: £399.97 - £483.76
    Return range: £446.89 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:      32.4%
    IDM:       0.0%
    DC:       67.6%
    Expected Return: £ 738.01
    Portfolio Risk:  £ 411.48
    Utility (A=3):     0.242

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D* test profits stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'base_case','down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'green','lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=100):
    """Generate complete efficient frontier including both efficient and inefficient portions"""
    
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    min_ret = min(returns)
    
    # Create extended range that goes beyond minimum variance portfolio
    # This creates the full hyperbola, not just the efficient portion
    return_range_low = np.linspace(min_ret, min_return, n_portfolios//2)  # Inefficient portion
    return_range_high = np.linspace(min_return, max_ret, n_portfolios//2)  # Efficient portion
    
    # Combine both ranges, removing duplicate minimum variance point
    target_returns = np.concatenate([return_range_low[:-1], return_range_high])
    
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            # Mark whether this is efficient (above min var) or inefficient (below min var)
            is_efficient = portfolio_return >= min_return
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk, is_efficient])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 5 and current scenario
    config_data = df[(df['config_id'] == 5) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_5_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_5_scenario['dam_mean'], config_5_scenario['idm_mean'], config_5_scenario['dc_mean']])
    risks_original = np.array([config_5_scenario['dam_std'], config_5_scenario['idm_std'], config_5_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_5_scenario['corr_dam_idm'], config_5_scenario['corr_dam_dc']],
        [config_5_scenario['corr_dam_idm'], 1.000, config_5_scenario['corr_idm_dc']],
        [config_5_scenario['corr_dam_dc'], config_5_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate complete efficient frontier for this scenario
    complete_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(complete_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(complete_frontier[:, 1])
        
        # Separate efficient and inefficient portions
        efficient_mask = complete_frontier[:, 7].astype(bool)  # is_efficient flag
        efficient_portion = complete_frontier[efficient_mask]
        inefficient_portion = complete_frontier[~efficient_mask]
        
        # Plot EFFICIENT portion as solid line
        if len(efficient_portion) > 0:
            fig.add_trace(go.Scatter(
                x=efficient_portion[:, 1],  # Risk (normalized)
                y=efficient_portion[:, 0],  # Return (normalized)
                mode='lines',
                line=dict(color=scenario_colors[i], width=3),
                name=f'Efficient Frontier ({scenario})',
                showlegend=True,
                hoverinfo='skip'
            ))
            
            # Add dots for efficient portion
            fig.add_trace(go.Scatter(
                x=efficient_portion[:, 1],
                y=efficient_portion[:, 0],
                mode='markers',
                marker=dict(size=4, color=scenario_colors[i], opacity=0.8),
                name=f'Efficient Portfolios ({scenario})',
                showlegend=False,
                hovertemplate='<b>EFFICIENT Portfolio (' + scenario + '):</b><br>' +
                              'DAM: %{customdata[0]:.1%}<br>' +
                              'IDM: %{customdata[1]:.1%}<br>' +
                              'DC: %{customdata[2]:.1%}<br>' +
                              '<b>Original Scale:</b><br>' +
                              'Return: £%{customdata[3]:.2f}<br>' +
                              'Risk: £%{customdata[4]:.2f}<br>' +
                              '<extra></extra>',
                customdata=np.column_stack((efficient_portion[:, 2],  # DAM weight
                                           efficient_portion[:, 3],  # IDM weight
                                           efficient_portion[:, 4],  # DC weight
                                           efficient_portion[:, 5],  # Original return
                                           efficient_portion[:, 6])) # Original risk
            ))
        
        # Plot INEFFICIENT portion as dashed line
        if len(inefficient_portion) > 0:
            fig.add_trace(go.Scatter(
                x=inefficient_portion[:, 1],  # Risk (normalized)
                y=inefficient_portion[:, 0],  # Return (normalized)
                mode='lines',
                line=dict(color=scenario_colors[i], width=2, dash='dash'),
                name=f'Inefficient Frontier ({scenario})',
                showlegend=False,  # Don't clutter legend
                hoverinfo='skip'
            ))
            
            # Add dots for inefficient portion
            fig.add_trace(go.Scatter(
                x=inefficient_portion[:, 1],
                y=inefficient_portion[:, 0],
                mode='markers',
                marker=dict(size=3, color=scenario_colors[i], opacity=0.5, symbol='circle-open'),
                name=f'Inefficient Portfolios ({scenario})',
                showlegend=False,
                hovertemplate='<b>INEFFICIENT Portfolio (' + scenario + '):</b><br>' +
                              'DAM: %{customdata[0]:.1%}<br>' +
                              'IDM: %{customdata[1]:.1%}<br>' +
                              'DC: %{customdata[2]:.1%}<br>' +
                              '<b>Original Scale:</b><br>' +
                              'Return: £%{customdata[3]:.2f}<br>' +
                              'Risk: £%{customdata[4]:.2f}<br>' +
                              '<i>Note: Higher risk for same return</i><br>' +
                              '<extra></extra>',
                customdata=np.column_stack((inefficient_portion[:, 2],  # DAM weight
                                           inefficient_portion[:, 3],  # IDM weight
                                           inefficient_portion[:, 4],  # DC weight
                                           inefficient_portion[:, 5],  # Original return
                                           inefficient_portion[:, 6])) # Original risk
            ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio using circle marker (dot)
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='circle', size=10, color=scenario_colors[i], line=dict(width=2, color='white')),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': complete_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 5: COMPLETE Efficient Frontiers for Different DC Scenarios<br><sub>Showing Full Hyperbola: Efficient (solid) & Inefficient (dashed) Portions | Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("COMPLETE EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 5")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Count efficient vs inefficient portfolios
        ef = data['efficient_frontier']
        if len(ef) > 0 and ef.shape[1] > 7:
            efficient_count = np.sum(ef[:, 7].astype(bool))
            inefficient_count = len(ef) - efficient_count
            
            print(f"  Complete Frontier:")
            print(f"    Total Portfolios: {len(ef)}")
            print(f"    Efficient Portfolios: {efficient_count}")
            print(f"    Inefficient Portfolios: {inefficient_count}")
            print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
            print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


COMPLETE EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 5

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:       0.0%
    DC:      100.0%
    Expected Return: £1037.23
    Portfolio Risk:  £ 524.07
    Utility (A=3):     1.399
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolios: 49
    Inefficient Portfolios: 50
    Risk range: £522.69 - £579.03
    Return range: £649.12 - £1037.23

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:       0.0%
    DC:      100.0%
    Expected Return: £ 957.44
    Portfolio Risk:  £ 529.53
    Utility (A=3):     1.084
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolios: 49
    Inefficient Portfolios: 50
    Risk range: £523.37 - £552.15
    Return range: £649.12 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      16.8%
    DC:       83.2%
    Expected Return: £ 844.65
    Portfolio Risk:  £ 530.18
    Utility (A=3):     0.472

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go

# Load the CSV data
df = pd.read_csv('/Users/haixiaosun/Library/Mobile Documents/com~apple~CloudDocs/Coding Work/Markowitz exploration/Results EDA/Results for EDA/D* test profits stats.csv')

# Define scenarios to analyze
scenarios = ['up_30%', 'up_20%', 'up_10%', 'base_case','down_10%', 'down_20%', 'down_30%']
scenario_colors = ['darkred', 'red', 'orange', 'green','lightblue', 'blue', 'darkblue']

def calculate_portfolio_stats(weights, returns, cov_matrix):
    """Calculate return and risk for given weights"""
    portfolio_return = np.dot(weights, returns)
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_return, portfolio_risk, portfolio_variance

def find_minimum_variance_portfolio(returns, cov_matrix):
    """Find the global minimum variance portfolio"""
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    
    if result.success:
        weights = result.x
        portfolio_return = np.dot(weights, returns)
        portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
        return weights, portfolio_return, portfolio_risk
    return None

def generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original, n_portfolios=100):
    """Generate complete efficient frontier including both efficient and inefficient portions"""
    
    # Find minimum variance portfolio
    min_var_result = find_minimum_variance_portfolio(returns, cov_matrix)
    if min_var_result is None:
        return np.array([])
    
    _, min_return, min_risk = min_var_result
    max_ret = max(returns)
    min_ret = min(returns)
    
    # Create extended range that goes beyond minimum variance portfolio
    # This creates the full hyperbola, not just the efficient portion
    return_range_low = np.linspace(min_ret, min_return, n_portfolios//2)  # Inefficient portion
    return_range_high = np.linspace(min_return, max_ret, n_portfolios//2)  # Efficient portion
    
    # Combine both ranges, removing duplicate minimum variance point
    target_returns = np.concatenate([return_range_low[:-1], return_range_high])
    
    efficient_portfolios = []
    
    for target_return in target_returns:
        # Minimize risk for given return
        def objective(weights):
            return np.dot(weights, np.dot(cov_matrix, weights))
        
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w, returns) - target_return}  # achieve target return
        ]
        
        bounds = [(0, 1), (0, 1), (0, 1)]
        
        result = minimize(objective, x0=[1/3, 1/3, 1/3],
                        bounds=bounds, constraints=constraints, method='SLSQP')
        
        if result.success:
            weights = result.x
            portfolio_return = np.dot(weights, returns)
            portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
            
            # Convert to original scale for storage
            original_return = portfolio_return * returns_original.std() + returns_original.mean()
            original_risk = portfolio_risk * risks_original.std() + risks_original.mean()
            
            # Mark whether this is efficient (above min var) or inefficient (below min var)
            is_efficient = portfolio_return >= min_return
            
            efficient_portfolios.append([portfolio_return, portfolio_risk, weights[0], weights[1], weights[2],
                                       original_return, original_risk, is_efficient])
    
    return np.array(efficient_portfolios)

def find_best_portfolio(risk_aversion, returns, cov_matrix):
    """Find portfolio that maximizes utility"""
    def objective_function(weights):
        port_return, _, port_variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        utility = port_return - 0.5 * risk_aversion * port_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    result = minimize(objective_function, x0=[1/3, 1/3, 1/3], bounds=bounds, constraints=constraints)
    best_weights = result.x
    best_return, best_risk, best_variance = calculate_portfolio_stats(best_weights, returns, cov_matrix)
    
    return best_weights, best_return, best_risk

# Create interactive plot
fig = go.Figure()

# Store data for each scenario
scenario_data = {}

# Collect all efficient frontiers to determine risk range
all_risks = []

# Process each scenario
for i, scenario in enumerate(scenarios):
    # Filter for Config 8 and current scenario
    config_data = df[(df['config_id'] == 8) & (df['scenario'] == scenario)]
    
    if config_data.empty:
        print(f"Warning: No data found for scenario {scenario}")
        continue
        
    config_8_scenario = config_data.iloc[0]
    
    # Extract market data from CSV
    returns_original = np.array([config_8_scenario['dam_mean'], config_8_scenario['idm_mean'], config_8_scenario['dc_mean']])
    risks_original = np.array([config_8_scenario['dam_std'], config_8_scenario['idm_std'], config_8_scenario['dc_std']])
    
    # Normalize returns and risks (mean=0, std=1)
    returns = (returns_original - returns_original.mean()) / returns_original.std()
    risks = (risks_original - risks_original.mean()) / risks_original.std()
    
    # Build correlation matrix from CSV data
    correlations = np.array([
        [1.000, config_8_scenario['corr_dam_idm'], config_8_scenario['corr_dam_dc']],
        [config_8_scenario['corr_dam_idm'], 1.000, config_8_scenario['corr_idm_dc']],
        [config_8_scenario['corr_dam_dc'], config_8_scenario['corr_idm_dc'], 1.000]
    ])
    
    # Calculate covariance matrix with normalized data
    cov_matrix = np.outer(risks, risks) * correlations
    
    # Generate complete efficient frontier for this scenario
    complete_frontier = generate_efficient_frontier(returns, cov_matrix, returns_original, risks_original)
    
    if len(complete_frontier) > 0:
        # Store risk values for range calculation
        all_risks.extend(complete_frontier[:, 1])
        
        # Separate efficient and inefficient portions
        efficient_mask = complete_frontier[:, 7].astype(bool)  # is_efficient flag
        efficient_portion = complete_frontier[efficient_mask]
        inefficient_portion = complete_frontier[~efficient_mask]
        
        # Plot EFFICIENT portion as solid line
        if len(efficient_portion) > 0:
            fig.add_trace(go.Scatter(
                x=efficient_portion[:, 1],  # Risk (normalized)
                y=efficient_portion[:, 0],  # Return (normalized)
                mode='lines',
                line=dict(color=scenario_colors[i], width=3),
                name=f'Efficient Frontier ({scenario})',
                showlegend=True,
                hoverinfo='skip'
            ))
            
            # Add dots for efficient portion
            fig.add_trace(go.Scatter(
                x=efficient_portion[:, 1],
                y=efficient_portion[:, 0],
                mode='markers',
                marker=dict(size=4, color=scenario_colors[i], opacity=0.8),
                name=f'Efficient Portfolios ({scenario})',
                showlegend=False,
                hovertemplate='<b>EFFICIENT Portfolio (' + scenario + '):</b><br>' +
                              'DAM: %{customdata[0]:.1%}<br>' +
                              'IDM: %{customdata[1]:.1%}<br>' +
                              'DC: %{customdata[2]:.1%}<br>' +
                              '<b>Original Scale:</b><br>' +
                              'Return: £%{customdata[3]:.2f}<br>' +
                              'Risk: £%{customdata[4]:.2f}<br>' +
                              '<extra></extra>',
                customdata=np.column_stack((efficient_portion[:, 2],  # DAM weight
                                           efficient_portion[:, 3],  # IDM weight
                                           efficient_portion[:, 4],  # DC weight
                                           efficient_portion[:, 5],  # Original return
                                           efficient_portion[:, 6])) # Original risk
            ))
        
        # Plot INEFFICIENT portion as dashed line
        if len(inefficient_portion) > 0:
            fig.add_trace(go.Scatter(
                x=inefficient_portion[:, 1],  # Risk (normalized)
                y=inefficient_portion[:, 0],  # Return (normalized)
                mode='lines',
                line=dict(color=scenario_colors[i], width=2, dash='dash'),
                name=f'Inefficient Frontier ({scenario})',
                showlegend=False,  # Don't clutter legend
                hoverinfo='skip'
            ))
            
            # Add dots for inefficient portion
            fig.add_trace(go.Scatter(
                x=inefficient_portion[:, 1],
                y=inefficient_portion[:, 0],
                mode='markers',
                marker=dict(size=3, color=scenario_colors[i], opacity=0.5, symbol='circle-open'),
                name=f'Inefficient Portfolios ({scenario})',
                showlegend=False,
                hovertemplate='<b>INEFFICIENT Portfolio (' + scenario + '):</b><br>' +
                              'DAM: %{customdata[0]:.1%}<br>' +
                              'IDM: %{customdata[1]:.1%}<br>' +
                              'DC: %{customdata[2]:.1%}<br>' +
                              '<b>Original Scale:</b><br>' +
                              'Return: £%{customdata[3]:.2f}<br>' +
                              'Risk: £%{customdata[4]:.2f}<br>' +
                              '<i>Note: Higher risk for same return</i><br>' +
                              '<extra></extra>',
                customdata=np.column_stack((inefficient_portion[:, 2],  # DAM weight
                                           inefficient_portion[:, 3],  # IDM weight
                                           inefficient_portion[:, 4],  # DC weight
                                           inefficient_portion[:, 5],  # Original return
                                           inefficient_portion[:, 6])) # Original risk
            ))
        
        # Find and plot optimal portfolio for A=3
        A = 3
        weights, expected_return, risk = find_best_portfolio(A, returns, cov_matrix)
        _, _, variance = calculate_portfolio_stats(weights, returns, cov_matrix)
        opt_utility = expected_return - 0.5 * A * variance
        
        # Convert to original scale
        original_return = expected_return * returns_original.std() + returns_original.mean()
        original_risk = risk * risks_original.std() + risks_original.mean()
        
        # Create indifference curve: E[R] = U + 0.5*A*σ²
        risk_min = max(0, min(all_risks) if all_risks else 0)
        risk_max = max(all_risks) if all_risks else 2
        risk_range = np.linspace(risk_min, risk_max * 1.5, 200)
        variance_range = risk_range ** 2
        indifference_curve = opt_utility + 0.5 * A * variance_range
        
        # Plot thin indifference curve
        fig.add_trace(go.Scatter(
            x=risk_range,
            y=indifference_curve,
            mode='lines',
            line=dict(color=scenario_colors[i], width=1, dash='dot'),
            name=f'Utility Curve ({scenario})',
            showlegend=False,
            hovertemplate='<b>Utility Curve (' + scenario + '):</b><br>' +
                          'U = E(r) - 0.5 × 3 × σ²<br>' +
                          'Utility = ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Plot optimal portfolio using circle marker (dot)
        fig.add_trace(go.Scatter(
            x=[risk],
            y=[expected_return],
            mode='markers',
            marker=dict(symbol='circle', size=10, color=scenario_colors[i], line=dict(width=2, color='white')),
            name=f'Optimal A=3 ({scenario})',
            hovertemplate='<b>Optimal Portfolio A=3 (' + scenario + '):</b><br>' +
                          'DAM: ' + f'{weights[0]:.1%}' + '<br>' +
                          'IDM: ' + f'{weights[1]:.1%}' + '<br>' +
                          'DC: ' + f'{weights[2]:.1%}' + '<br>' +
                          '<b>Original Scale:</b><br>' +
                          'Return: £' + f'{original_return:.2f}' + '<br>' +
                          'Risk: £' + f'{original_risk:.2f}' + '<br>' +
                          '<b>Utility:</b> ' + f'{opt_utility:.3f}' + '<br>' +
                          '<extra></extra>'
        ))
        
        # Store scenario data
        scenario_data[scenario] = {
            'efficient_frontier': complete_frontier,
            'optimal_weights': weights,
            'optimal_return': original_return,
            'optimal_risk': original_risk,
            'optimal_utility': opt_utility,
            'returns_original': returns_original,
            'risks_original': risks_original
        }

# Update layout
fig.update_layout(
    title='Config 8: COMPLETE Efficient Frontiers for Different DC Scenarios<br><sub>Showing Full Hyperbola: Efficient (solid) & Inefficient (dashed) Portions | Risk Aversion A=3</sub>',
    xaxis_title='Portfolio Risk (Normalized Standard Deviation)',
    yaxis_title='Portfolio Expected Return (Normalized)',
    width=1200,
    height=800,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show(renderer="browser")

# Print summary results
print("\n" + "="*80)
print("COMPLETE EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 8")
print("="*80)

for scenario in scenarios:
    if scenario in scenario_data:
        data = scenario_data[scenario]
        print(f"\n{scenario.upper()} SCENARIO:")
        print(f"  Optimal Portfolio (A=3):")
        print(f"    DAM:     {data['optimal_weights'][0]:6.1%}")
        print(f"    IDM:     {data['optimal_weights'][1]:6.1%}")
        print(f"    DC:      {data['optimal_weights'][2]:6.1%}")
        print(f"    Expected Return: £{data['optimal_return']:7.2f}")
        print(f"    Portfolio Risk:  £{data['optimal_risk']:7.2f}")
        print(f"    Utility (A=3):   {data['optimal_utility']:7.3f}")
        
        # Count efficient vs inefficient portfolios
        ef = data['efficient_frontier']
        if len(ef) > 0 and ef.shape[1] > 7:
            efficient_count = np.sum(ef[:, 7].astype(bool))
            inefficient_count = len(ef) - efficient_count
            
            print(f"  Complete Frontier:")
            print(f"    Total Portfolios: {len(ef)}")
            print(f"    Efficient Portfolios: {efficient_count}")
            print(f"    Inefficient Portfolios: {inefficient_count}")
            print(f"    Risk range: £{min(ef[:, 6]):.2f} - £{max(ef[:, 6]):.2f}")
            print(f"    Return range: £{min(ef[:, 5]):.2f} - £{max(ef[:, 5]):.2f}")


COMPLETE EFFICIENT FRONTIER ANALYSIS FOR DC SCENARIOS - CONFIG 8

UP_30% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      29.4%
    DC:       70.6%
    Expected Return: £ 992.25
    Portfolio Risk:  £ 697.58
    Utility (A=3):     0.137
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolios: 49
    Inefficient Portfolios: 50
    Risk range: £645.15 - £737.59
    Return range: £845.65 - £1037.23

UP_20% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      36.0%
    DC:       64.0%
    Expected Return: £ 931.13
    Portfolio Risk:  £ 693.93
    Utility (A=3):     0.069
  Complete Frontier:
    Total Portfolios: 99
    Efficient Portfolios: 50
    Inefficient Portfolios: 49
    Risk range: £623.22 - £751.02
    Return range: £845.65 - £957.44

UP_10% SCENARIO:
  Optimal Portfolio (A=3):
    DAM:       0.0%
    IDM:      54.1%
    DC:       45.9%
    Expected Return: £ 881.27
    Portfolio Risk:  £ 685.95
    Utility (A=3):     0.108